In [6]:
# Copyright 2015 The TensorFlow Authors. All Rights Reserved.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
# ==============================================================================
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import collections
import math
import os
import random
import zipfile

import numpy as np
from six.moves import urllib
from six.moves import xrange  # pylint: disable=redefined-builtin
import tensorflow as tf

# Step 1: Download the data.
url = 'http://mattmahoney.net/dc/'


def maybe_download(filename, expected_bytes):
  """Download a file if not present, and make sure it's the right size."""
  if not os.path.exists(filename):
    filename, _ = urllib.request.urlretrieve(url + filename, filename)
  statinfo = os.stat(filename)
  if statinfo.st_size == expected_bytes:
    print('Found and verified', filename)
  else:
    print(statinfo.st_size)
    raise Exception(
        'Failed to verify ' + filename + '. Can you get to it with a browser?')
  return filename

filename = maybe_download('text8.zip', 31344016)

Found and verified text8.zip


In [10]:
# Read the data into a list of strings.
def read_data(filename):
  """Extract the first file enclosed in a zip file as a list of words"""
  with zipfile.ZipFile(filename) as f:
    s = f.read(f.namelist()[0])
    data = tf.compat.as_str(s).split()
  return data

words = read_data(filename)

In [1]:
import pandas as pd
import re
from tqdm import tqdm
import nltk

In [2]:
s = pd.read_csv("yelp_academic_dataset_review_train.csv")

In [3]:
all_words = []
regex = re.compile('[^a-zA-Z]')
for index, item in tqdm(s.iterrows()):
    new_words = item['text'].split();
    new_words = [regex.sub('', w).lower() for w in new_words]
    all_words += new_words

116474it [00:20, 5706.54it/s]


In [4]:
words = all_words

11527274

In [7]:
print('Data size', len(words))

# Step 2: Build the dictionary and replace rare words with UNK token.
vocabulary_size = 5000

def build_dataset(words, vocabulary_size):
  count = [['UNK', -1]]
  count.extend(collections.Counter(words).most_common(vocabulary_size - 1))
  dictionary = dict()
  for word, _ in count:
    dictionary[word] = len(dictionary)
  data = list()
  unk_count = 0
  for word in words:
    if word in dictionary:
      index = dictionary[word]
    else:
      index = 0  # dictionary['UNK']
      unk_count += 1
    data.append(index)
  count[0][1] = unk_count
  reverse_dictionary = dict(zip(dictionary.values(), dictionary.keys()))
  return data, count, dictionary, reverse_dictionary

data, count, dictionary, reverse_dictionary = build_dataset(words, vocabulary_size)
del words  # Hint to reduce memory.
print('Most common words (+UNK)', count[:5])
print('Sample data', data[:10], [reverse_dictionary[i] for i in data[:10]])

data_index = 0




Data size 11527274
Most common words (+UNK) [['UNK', 419292], ('the', 587834), ('and', 393327), ('i', 311826), ('a', 287442)]
Sample data [3, 330, 88, 1122, 22, 45, 7, 1, 746, 69] ['i', 'almost', 'got', 'sick', 'on', 'all', 'of', 'the', 'chocolate', 'coffee']


In [8]:
# Step 3: Function to generate a training batch for the skip-gram model.
def generate_batch(batch_size, num_skips, skip_window):
  global data_index
  assert batch_size % num_skips == 0
  assert num_skips <= 2 * skip_window
  batch = np.ndarray(shape=(batch_size), dtype=np.int32)
  labels = np.ndarray(shape=(batch_size, 1), dtype=np.int32)
  span = 2 * skip_window + 1  # [ skip_window target skip_window ]
  buffer = collections.deque(maxlen=span)
  for _ in range(span):
    buffer.append(data[data_index])
    data_index = (data_index + 1) % len(data)
  for i in range(batch_size // num_skips):
    target = skip_window  # target label at the center of the buffer
    targets_to_avoid = [skip_window]
    for j in range(num_skips):
      while target in targets_to_avoid:
        target = random.randint(0, span - 1)
      targets_to_avoid.append(target)
      batch[i * num_skips + j] = buffer[skip_window]
      labels[i * num_skips + j, 0] = buffer[target]
    buffer.append(data[data_index])
    data_index = (data_index + 1) % len(data)
  # Backtrack a little bit to avoid skipping words in the end of a batch
  data_index = (data_index + len(data) - span) % len(data)
  return batch, labels

batch, labels = generate_batch(batch_size=8, num_skips=2, skip_window=1)
for i in range(8):
  print(batch[i], reverse_dictionary[batch[i]],
        '->', labels[i, 0], reverse_dictionary[labels[i, 0]])



330 almost -> 88 got
330 almost -> 3 i
88 got -> 1122 sick
88 got -> 330 almost
1122 sick -> 22 on
1122 sick -> 88 got
22 on -> 45 all
22 on -> 1122 sick


In [9]:
# Step 4: Build and train a skip-gram model.

batch_size = 128
embedding_size = 32  # Dimension of the embedding vector.
skip_window = 1       # How many words to consider left and right.
num_skips = 2         # How many times to reuse an input to generate a label.

# We pick a random validation set to sample nearest neighbors. Here we limit the
# validation samples to the words that have a low numeric ID, which by
# construction are also the most frequent.
valid_size = 16     # Random set of words to evaluate similarity on.
valid_window = 100  # Only pick dev samples in the head of the distribution.
valid_examples = np.random.choice(valid_window, valid_size, replace=False)
num_sampled = 64    # Number of negative examples to sample.

graph = tf.Graph()

with graph.as_default():

  # Input data.
  train_inputs = tf.placeholder(tf.int32, shape=[batch_size])
  train_labels = tf.placeholder(tf.int32, shape=[batch_size, 1])
  valid_dataset = tf.constant(valid_examples, dtype=tf.int32)

  # Ops and variables pinned to the CPU because of missing GPU implementation
  with tf.device('/cpu:0'):
    # Look up embeddings for inputs.
    embeddings = tf.Variable(
        tf.random_uniform([vocabulary_size, embedding_size], -1.0, 1.0))
    embed = tf.nn.embedding_lookup(embeddings, train_inputs)

    # Construct the variables for the NCE loss
    nce_weights = tf.Variable(
        tf.truncated_normal([vocabulary_size, embedding_size],
                            stddev=1.0 / math.sqrt(embedding_size)))
    nce_biases = tf.Variable(tf.zeros([vocabulary_size]))

  # Compute the average NCE loss for the batch.
  # tf.nce_loss automatically draws a new sample of the negative labels each
  # time we evaluate the loss.
  loss = tf.reduce_mean(
      tf.nn.nce_loss(weights=nce_weights,
                     biases=nce_biases,
                     labels=train_labels,
                     inputs=embed,
                     num_sampled=num_sampled,
                     num_classes=vocabulary_size))

  # Construct the SGD optimizer using a learning rate of 1.0.
  optimizer = tf.train.GradientDescentOptimizer(1.0).minimize(loss)

  # Compute the cosine similarity between minibatch examples and all embeddings.
  norm = tf.sqrt(tf.reduce_sum(tf.square(embeddings), 1, keep_dims=True))
  normalized_embeddings = embeddings / norm
  valid_embeddings = tf.nn.embedding_lookup(
      normalized_embeddings, valid_dataset)
  similarity = tf.matmul(
      valid_embeddings, normalized_embeddings, transpose_b=True)

  # Add variable initializer.
  init = tf.global_variables_initializer()



In [10]:
# Step 5: Begin training.
num_steps = 50001

with tf.Session(graph=graph) as session:
  # We must initialize all variables before we use them.
  init.run()
  print("Initialized")

  average_loss = 0
  for step in xrange(num_steps):
    batch_inputs, batch_labels = generate_batch(
        batch_size, num_skips, skip_window)
    feed_dict = {train_inputs: batch_inputs, train_labels: batch_labels}

    # We perform one update step by evaluating the optimizer op (including it
    # in the list of returned values for session.run()
    _, loss_val = session.run([optimizer, loss], feed_dict=feed_dict)
    average_loss += loss_val

    if step % 2000 == 0:
      if step > 0:
        average_loss /= 2000
      # The average loss is an estimate of the loss over the last 2000 batches.
      print("Average loss at step ", step, ": ", average_loss)
      average_loss = 0

    # Note that this is expensive (~20% slowdown if computed every 500 steps)
    if step % 10000 == 0:
      sim = similarity.eval()
      for i in xrange(valid_size):
        valid_word = reverse_dictionary[valid_examples[i]]
        top_k = 8  # number of nearest neighbors
        nearest = (-sim[i, :]).argsort()[1:top_k + 1]
        log_str = "Nearest to %s:" % valid_word
        for k in xrange(top_k):
          close_word = reverse_dictionary[nearest[k]]
          log_str = "%s %s," % (log_str, close_word)
        print(log_str)
  final_embeddings = normalized_embeddings.eval()



Initialized
Average loss at step  0 :  183.761627197
Nearest to for: joy, overrated, brutta, taking, obnoxious, age, easily, generals,
Nearest to try: bistro, back, potential, margherita, cared, chilled, drenched, requests,
Nearest to out: in, szechwan, oreganos, move, floors, blame, phoenix, superior,
Nearest to by: measure, persons, tastier, parking, plane, focused, thankfully, jack,
Nearest to in: move, chimis, staff, out, figured, miss, substance, onions,
Nearest to service: creative, asado, select, assured, heads, pinches, guest, writing,
Nearest to on: strictly, commonplace, partner, mentioning, ish, fault, baristas, unique,
Nearest to their: plantains, pizzeria, normally, personality, powder, turn, vinegar, tart,
Nearest to ive: squash, includes, satisfying, plentiful, coffeehouse, walnuts, bring, lousy,
Nearest to more: id, clean, risk, ever, ah, ground, stays, ok,
Nearest to you: greeting, accurate, nonetheless, bags, fusion, pot, myself, appeared,
Nearest to it: freshly, bage

In [11]:
# Step 6: Visualize the embeddings.


def plot_with_labels(low_dim_embs, labels, filename='tsne.png'):
  assert low_dim_embs.shape[0] >= len(labels), "More labels than embeddings"
  plt.figure(figsize=(18, 18))  # in inches
  for i, label in enumerate(labels):
    x, y = low_dim_embs[i, :]
    plt.scatter(x, y)
    plt.annotate(label,
                 xy=(x, y),
                 xytext=(5, 2),
                 textcoords='offset points',
                 ha='right',
                 va='bottom')

  plt.savefig(filename)






In [13]:
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt

tsne = TSNE(perplexity=30, n_components=2, init='pca', n_iter=5000)
plot_only = 500
low_dim_embs = tsne.fit_transform(final_embeddings[:plot_only, :])
labels = [reverse_dictionary[i] for i in xrange(plot_only)]
plot_with_labels(low_dim_embs, labels)


In [104]:
em = pd.DataFrame(final_embeddings)
l = pd.DataFrame(list(reverse_dictionary.values()))
word2vec = pd.concat([l, em], axis = 1)
word2vec.to_csv("word2vec.csv")

In [7]:
all_words = []
regex = re.compile('[^a-zA-Z]')
for index, item in tqdm(s.iterrows()):
    new_words = item['text'].split();
    new_words = [regex.sub('', w).lower() for w in new_words]
    all_words += new_words
all_words = []



116474it [00:12, 9419.25it/s] 


In [27]:
import numpy as np
a=list(s.iterrows())

In [28]:
len(a)

116474

NameError: name 'ls' is not defined

In [15]:
op = np.array([], dtype=np.int64).reshape(0,32*128)
vecs=[]
for index, item in tqdm(s.iterrows()):
    new_words = item['text'].split();
    new_words = [regex.sub('', w).lower() for w in new_words]
    if (len(new_words) > 128):
        new_words = new_words[0:128]
    else:
        new_words += ["" for _ in range(128 - len(new_words))]
    new_words = [w if w in dictionary.keys() else "UNK" for w in new_words]
    new_words_vec = np.array([final_embeddings[dictionary[w]] for w in new_words]).reshape(1,32*128)
    vecs += [new_words_vec]

116474it [00:48, 2378.46it/s]


In [16]:
op = np.vstack(vecs)

In [18]:
op.shape

(116474, 4096)

In [20]:
np.savetxt("embedded_sentence.csv", op, delimiter=",")

In [21]:
opa = np.array(op, dtype=np.int32)

In [22]:
np.savetxt("embedded_sentence.csv", opa, delimiter=",")

(9, 87552)

In [30]:
new_words_vec

[array([-0.03356166, -0.02698038,  0.0875643 , -0.03607975,  0.11286078,
         0.05114302, -0.00647421,  0.03907421, -0.05090344,  0.01008697,
         0.03860681, -0.00500363,  0.06882046, -0.06654992,  0.01815865,
         0.0078345 , -0.0179046 , -0.03511933, -0.03229256,  0.03310082,
         0.05127196, -0.07398817,  0.0268654 , -0.02175187, -0.03331941,
         0.16532724, -0.02248883, -0.09953922, -0.03464411, -0.00098884,
         0.04340502,  0.10713065,  0.06845216, -0.0987809 , -0.03930199,
        -0.02232163, -0.01610057, -0.10144462,  0.05493654,  0.05152472,
        -0.02266102, -0.05730942, -0.00306749, -0.0515701 , -0.02287491,
        -0.02554948,  0.08722634, -0.12052142, -0.05980081, -0.09106068,
         0.08536525, -0.02977733, -0.03220539,  0.05484799,  0.00238857,
        -0.05608227, -0.06866138,  0.10637245,  0.00826227, -0.00641244,
         0.00305132,  0.07190569, -0.07480074,  0.01298789,  0.05137379,
        -0.04983038,  0.01660604, -0.08601016, -0.0